In [ ]:
import os 
import re

import pyedflib
import numpy as np 
import matplotlib.pyplot as plt

import skfda
import skfda.representation.basis as basis
from skfda.exploratory.visualization import FPCAPlot
from skfda.preprocessing.dim_reduction import FPCA
from skfda.representation.basis import BSplineBasis
from skfda.representation.interpolation import SplineInterpolation

from scipy import stats 

#NOTE: dataset needs to be downloaded and unziped  

#filter edf files
eeg_signalfiles_names = list(filter(lambda s : s.endswith(".edf") , 
                            [os.path.abspath(os.path.join("../data/eeg-during-mental-arithmetic-tasks-1.0.0", f)) for 
                            f in os.listdir("../data/eeg-during-mental-arithmetic-tasks-1.0.0")]
                        ))

#dicts with subject_id - eeg_signal_filepath
before_arith_task = {}
after_arith_task = {}

# process each file
for edf_file_name in eeg_signalfiles_names :
    file_name = os.path.basename(edf_file_name)
    
    # extract subject identifier and suffix
    if "_1.edf" in file_name:
        subject_id = int(re.findall(r"\d\d", file_name)[0])
        before_arith_task[subject_id] = edf_file_name
    
    elif "_2.edf" in file_name:
        subject_id = int(re.findall(r"\d\d", file_name)[0])
        after_arith_task[subject_id] = edf_file_name

In [ ]:
print("len of dict bat: ",len(before_arith_task.keys()))
print("len of dict aat: ",len(after_arith_task.keys()))

In [ ]:
label_index_dict = {"Fp1":0,"Fp2":1,"F3":2,"F4":3,"F7":4,"F8":5,"T3":6,"T4":7,"C3":8,
                    "C4":9,"T5":10,"T6":11,"P3":12,"P4":13,"O1":14,"O2":15,"Fz":16, "Cz":17,"Pz":18}

In [ ]:
f = pyedflib.EdfReader(after_arith_task[10])
#print edf file metadata
print(f.file_info_long())
eeg_signal = f.readSignal(label_index_dict.get("P4", None))

f.close()
print()
print("eeg signal number 0: ", eeg_signal)

In [ ]:
def eeg_data_matrix(subject_idx, subject_edfsignal_dict, labels):
    """
    concatenate all eeg signals for contructing a 
    """
    if subject_edfsignal_dict.get(subject_idx,-1) == -1:
        raise KeyError
    #build data matrix
    with pyedflib.EdfReader(subject_edfsignal_dict[subject_idx]) as f:
        eeg_signals_arr = []
        for label in labels:
            idx = label_index_dict.get(label,-1)
            if idx == -1:
               raise KeyError
            eeg_signal = f.readSignal(idx)
            eeg_signals_arr.append(eeg_signal)
        f.close()
        return np.matrix(eeg_signals_arr)

In [ ]:
all_labels = list(label_index_dict.keys())
data = eeg_data_matrix(10,after_arith_task,all_labels)
print("data : ", data)

In [ ]:
#raw eeg-signals transformed into functional data using 
#B-splines basis functions

fd = skfda.FDataGrid(
            data_matrix=data,
            grid_points=np.linspace(0,31000,31000)/500
        )
n_basis = 19

fd.plot()

In [ ]:
#plot b-spline basis
bspline_basis = skfda.representation.basis.BSplineBasis(n_basis=n_basis)
bspline_basis.plot()
plt.show()

In [ ]:
#plot of the eeg signals converted to functional data
fd_basis = fd.to_basis(basis.BSplineBasis(n_basis=n_basis))

fd_basis.plot()

In [ ]:
fpca = FPCA(n_basis)
fpca.fit(fd_basis)
fpca.components_.plot()

eigvals = np.square(fpca.singular_values_)
print("singular vals: ", eigvals)

In [ ]:
#compute here intercept and slope


In [ ]:
#plot truncated eigspectrum


In [ ]:
# r2 and KS test
